### Hitta Riksdagsmän i andra/första kammaren i Gravstensinventeringen

* exempel Gunnar Helen Wikidata [Q376484](https://www.wikidata.org/wiki/Q376484) = Gravstensinventeringen [379758?pid=1](https://www.rotter.se/gsi/379758%3Fpid%3D1)

Idag körs ett forskningsprojekt [Välfärden analyseras](ss)/[pressrelease](https://www.umu.se/nyheter/5-miljoner-till-forskning-om-lankad-parlamentariska-data_11678700/) där en del är att identifiera alla personer som suttit i Riksdagen,. Min tanke är att hitta så många av dessa i Gravstensinventeringem

1. Enklaste varianten skapa en sök URL för varje person och manuellt leta
2. Bättre köra någon form av API om ni har det och även kontinuerligt ligga och kolla om det kommit in nya
3. Ännu bättre om Gravstensinventeringen länkar tillbaka till Wikidata --> ni skulle kunna länka till WIkipedia [artiklar om personerna](https://w.wiki/6bgL), [bild](https://w.wiki/6bgP) eller [kyrkogårdarna](sss)

Alla dom personerna finns i Wikidata idag med koppling Gravstensinventeringen 
* GITHUB [issue #22](https://github.com/salgo60/Gravstensinventeringen-Wikidata/issues/22)
* [SPARQL fråga första /andra kammaren](https://w.wiki/6bNs)
 * med länk [sv:Wikipedia](https://w.wiki/6bfq)



In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-04-18 15:15:53.841698


In [4]:
# Wikidata hämta alla Svenska Riksdagsman i första andra kammaren wd:Q33071890 wd:Q81531912 
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# https://w.wiki/6bLf
query = """#title: Tvpkammar riksdage
SELECT distinct ?person ?personLabel  ?birthDate ?diedDate  WHERE {

  VALUES ?member {
    wd:Q33071890 
    wd:Q81531912 
  }
  ?person wdt:P39 ?member.
  ?person wdt:P569 ?birthDate. 
  ?person wdt:P570 ?diedDate
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
} order by ?personLabel"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def getDateFormatted(date_string):
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    parsed_date = datetime.strptime(date_string, date_format)

    formatted_date = parsed_date.strftime("%Y-%m-%d")
    return formatted_date

results = get_results(endpoint_url, query)

print ("namn","|", "birth", "|" ,"dead", "|" , "link")

for result in results["results"]["bindings"][1:10]:
    link = result["person"]["value"]
    name = result["personLabel"]["value"]
    birth = getDateFormatted(result["birthDate"]["value"])
    dead = getDateFormatted(result["diedDate"]["value"])
    print (name,"|", birth, "|" ,dead, "|" , link)


namn | birth | dead | link
Abel Andersson | 1887-06-15 | 1964-04-30 | http://www.wikidata.org/entity/Q5553810
Abel Sundberg | 1893-01-02 | 1974-12-22 | http://www.wikidata.org/entity/Q6195368
Abel Vinborg | 1835-09-17 | 1914-04-12 | http://www.wikidata.org/entity/Q6225637
Abraham Berglund | 1831-03-17 | 1905-03-22 | http://www.wikidata.org/entity/Q105065199
Abraham Leijonhufvud | 1823-03-31 | 1911-08-31 | http://www.wikidata.org/entity/Q5945036
Abraham Rundbäck | 1827-01-30 | 1893-06-12 | http://www.wikidata.org/entity/Q6083434
Adam Christian Raab | 1801-08-11 | 1872-06-08 | http://www.wikidata.org/entity/Q6060104
Adam Hult | 1870-12-11 | 1947-10-18 | http://www.wikidata.org/entity/Q5816342
Adam Reuterskiöld | 1819-06-08 | 1880-02-12 | http://www.wikidata.org/entity/Q6068925


In [ ]:
for result in results["results"]["bindings"][1:10]:
    link = result["person"]["value"]
    name = result["personLabel"]["value"]
    birth = getDateFormatted(result["birthDate"]["value"])
    dead = getDateFormatted(result["diedDate"]["value"])
    print (name,"|", birth, "|" ,dead, "|" , link)

## koppla dessa personer till Gravstensinventeringen

1. enkel variant är bara att skapa bra sökURL:ar med förnamn, födelseår 
   1. sedan manuellt klicka sig igenm detta
2. om det finns ett API ---> vi kan manuellt hitta detta
   1. kunna köra det med jämna mellanrum och se om några nya kandidater dyker upp
   1. skickar email och kollar vad som finns.... och deras tankar kanske kan dom länka tillbaka till Wikidata
   1. Jämför SPA Svenskt Porträttas API där vi hämtar alla personer kopplade till Norra begravningsplatsen och ser om vild finns - [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20test.ipynb)
      1. deras API har en snygg rank funktion som anger hur sannolik en träff är

In [3]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ass.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-04-18 15:15:58.888447
Time elapsed (hh:mm:ass.ms) 0:00:05.046938
